In [2]:
import json
import requests
import pandas as pd
from urllib.parse import urlencode, quote_plus
# https://stackoverflow.com/questions/40557606/how-to-url-encode-in-python-3
from bs4 import BeautifulSoup
from tqdm import tqdm


### SERP Crawler
#### rule by top domains
(6,7,8,9,10名都是公司網站首頁，應該不能跟文章優化放在一起比較?)

- da-vinci.com.tw
    - class: model05
- inboundmarketing.com.tw
    - class: elementor-widget-theme-post-content    
- yesharris.com
    - class: td-post-content
- growthmarketing.tw
    - class: entry-content
- ranking.works
    - class: article_content
- awoo.com.tw
    - class: elementor-widget-container


[首頁類] 乾脆就是全抓
- airnet.com.tw
    - class: elementor-section-wrap
- seo.tenten.co
    - class: gdlr-core-page-builder-body
- seo.whoops.com.tw
    - class: elementor-section-wrap
- seookay.com
    - id: wrapper
- taipeiads.com
    - body
- corners.com.tw
    - class: site-main
- pineapple-web.com
    - class: content
- leononline.com.tw
    - class: elementor-section-wrap
- sharktech.tw
    - class: 
- seo-seo.com.tw
- iwangoweb.com
- appleseo.com.tw
- iware.com.tw
- ibest.tw
- teng.com.tw
- leftmoon.co

有的是首頁、有的是服務介紹頁、有的是引導式文章(引導預約諮詢)
同個網域下，這三種網頁架構也不同，很難全部人工標好要抓的 css rule

網頁上所有文字都抓的合理說法：

對搜尋引擎來說，也是因為他認為會搜尋「seo公司」的 user 想看這樣的首頁內容，才會出現這些網頁；
就算會有跟seo服務、知識無關的雜訊字詞 (ex: 關於我們、聯絡我們)，如果所有網站都抓了，他們也會相對變得不重要


#### BeautifulSoup selector
https://www.itread01.com/content/1536692545.html


In [4]:
base_url = 'https://www.google.com/search?'
search_keyword = 'seo 公司'
num = 50
headers = {
    'content-type': 'text/html; charset=UTF-8',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}

params = {'q': search_keyword, 'num': num}
target_url = base_url + urlencode(params, quote_via=quote_plus)
print(target_url)


https://www.google.com/search?q=seo+%E5%85%AC%E5%8F%B8&num=50


In [18]:
collect = list()

html = requests.get(target_url, headers=headers, timeout=15).content
soup = BeautifulSoup(html, 'html.parser')
# print(soup.prettify())
for block in soup.find_all('div', class_='g'):
    title = block.h3.text
    desc = block.find('span', class_='st').text
    link = block.find('a').attrs['href']
    collect.append([title, desc, link])

df = pd.DataFrame(collect, columns=['title', 'desc', 'link'])
print(df.shape)
df.head()


(50, 3)


,title,desc,link
0,SEO收費行情？該怎麼選擇SEO公司?-SEO優化-部落格-達文西 ...,2020年3月17日 - 大家都在討論SEO關鍵字排名，市場報價非常紊亂，比網頁設計更沒有報...,https://www.da-vinci.com.tw/tw/blog/seo-cost
1,【SEO關鍵字公司】SEO公司怎麼選？看這篇推薦最適合你的 ...,一個合格的SEO公司能為你服務的，而不是只是單純的排名與成效收費，傳統類型的SEO 公司更重...,https://inboundmarketing.com.tw/seo/seo%E5%85%...
2,行銷人必讀，找SEO顧問前你該知道的事| Harris先生,以SEO顧問公司來說，即便顧問公司的顧問有行銷專業、網站分析專業、廣告投放的專業，你跑去問他...,https://www.yesharris.com/before-seo-agency/
3,SEO 公司怎麼選才有效？搞懂SEO 公司服務類型、收費以及 ...,2020年7月28日 - 想做SEO 的品牌，無非是想要獲得更多「自然搜尋流量」，讓更多潛在...,https://growthmarketing.tw/seo%E5%84%AA%E5%8C%...
4,【SEO觀念7】SEO 公司挑選指南：認識SEO 顧問與優化服務 ...,2019年8月18日 - 不同階段的企業可以藉由SEO 達到什麼樣的效益、服務有哪些形式以及...,https://ranking.works/SEO%E6%95%99%E5%AD%B8/SE...


In [21]:
df.to_csv('SERP_{}_num_{}.csv'.format(search_keyword, num), index=False)


In [12]:
# df = pd.read_csv('SERP_{}_num_{}.csv'.format(search_keyword, num))
[x for x in df.link.values if x.count('/') > 3]


['https://www.da-vinci.com.tw/tw/blog/seo-cost',
 'https://inboundmarketing.com.tw/seo/seo%E5%85%AC%E5%8F%B8%E6%8E%A8%E8%96%A6.html',
 'https://www.yesharris.com/before-seo-agency/',
 'https://growthmarketing.tw/seo%E5%84%AA%E5%8C%96%E5%85%AC%E5%8F%B8/',
 'https://ranking.works/SEO%E6%95%99%E5%AD%B8/SEO%E5%85%AC%E5%8F%B8',
 'https://www.airnet.com.tw/seo%E5%85%AC%E5%8F%B8/',
 'https://www.awoo.com.tw/blog/seo-company/',
 'https://www.pineapple-web.com/tw/seo-agency/',
 'https://www.asiapacdigital.com/zh-cht/SEO',
 'https://www.newscan.com.tw/all-seo/seo-guide.htm',
 'https://medium.com/@u78150722/seo%E5%85%AC%E5%8F%B8%E9%80%99%E5%80%91%E5%A4%9A%E5%93%AA%E5%80%8B%E6%89%8D%E6%98%AF%E5%A5%BD%E7%9A%84-3%E5%80%8B%E9%81%B8%E6%93%87%E9%87%8D%E9%BB%9E%E5%91%8A%E8%A8%B4%E4%BD%A0-ea1cbe5da2ea',
 'https://www.104.com.tw/job/6wi1a',
 'https://meethub.bnext.com.tw/%E3%80%902019%E6%9C%80%E6%96%B0%E3%80%91seo%E8%A6%81%E6%80%8E%E9%BA%BC%E5%81%9A%EF%BC%9Fseo%E6%93%8D%E4%BD%9C%E6%95%88%E7%9B%8A%E8%88%87

In [149]:
collect = list()

for link in tqdm(df.link.values):
# link = df.link.values[0]
    html = requests.get(link, headers=headers, timeout=15).content
    soup = BeautifulSoup(html, 'html.parser')
    try:
        article_desc = soup.find_all(attrs={'name': 'description'})[0].attrs['content']
    except:
        article_desc = ''

    try:
        article_keywords = soup.find_all(attrs={'name': 'keyword'})[0].attrs['content']
    except:
        article_keywords = ''
    
    # https://stackoverflow.com/questions/22799990/beatifulsoup4-get-text-still-has-javascript
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.decompose()    # rip it out
    # get text
    text = soup.body.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    article_content = ' '.join(chunk for chunk in chunks if chunk)
    
#     line_collect = list()
#     for line in soup.body.stripped_strings:
#         line_collect.append(line)
#     article_content = ' '.join(line_collect)

    try:
        article_published_time = soup.find_all(attrs={'property': 'article:published_time'})[0].attrs['content']
    except:
        article_published_time = ''

    try:
        article_modified_time = soup.find_all(attrs={'property': 'article:modified_time'})[0].attrs['content']
    except:
        article_modified_time = ''

    try:
        article_schema = soup.find_all(attrs={'type': 'application/ld+json'})[0].text
    except:
        article_schema = ''
    else:
        if (article_published_time == '') and ('datePublished' in article_schema):
            try:
                article_published_time = json.loads(article_schema)['@graph'][1]['datePublished']
            except:
                try:
                    article_published_time = json.loads(article_schema)[0]['datePublished']
                except:
                    try:
                        article_published_time = json.loads(article_schema)['@graph'][3]['datePublished']
                    except:
                        try:
                            article_published_time = json.loads(article_schema)['@graph'][2]['datePublished']
                        except:
                            try:
                                article_published_time = json.loads(article_schema)['datePublished']
                            except:
                                article_published_time = json.loads(article_schema)['@graph'][0]['datePublished']
                                
                            
#                     print(json.loads(article_schema))

        if (article_modified_time == '') and ('dateModified' in article_schema):
            try:
                article_modified_time = json.loads(article_schema)['@graph'][1]['dateModified']
            except:
                try:
                    article_modified_time = json.loads(article_schema)[0]['dateModified']
                except:
                    try:
                        article_modified_time = json.loads(article_schema)['@graph'][3]['dateModified']
                    except:
                        try:
                            article_modified_time = json.loads(article_schema)['@graph'][2]['dateModified']
                        except:
                            try:
                                article_modified_time = json.loads(article_schema)['dateModified']
                            except:
                                article_modified_time = json.loads(article_schema)['@graph'][0]['dateModified']
                                

        if (article_keywords == '') and ('keywords' in article_schema):
            try:
                article_keywords = json.loads(article_schema)[0]['keywords']
            except:
                try:
                    article_keywords = json.loads(article_schema)['@graph'][5]['keywords']
                except:
                    article_keywords = json.loads(article_schema)['keywords']                    
#                 print(json.loads(article_schema))
    
    collect.append([link, article_desc, article_keywords, article_content, article_published_time, article_modified_time, article_schema])









  0%|                                                                                                                                                                                                                | 0/50 [00:00<?, ?it/s]





  2%|████                                                                                                                                                                                                    | 1/50 [00:00<00:08,  5.49it/s]





  4%|████████                                                                                                                                                                                                | 2/50 [00:00<00:09,  4.87it/s]





  6%|████████████                                                                                                                                                                                            | 3/50 [00:00<00:08,  5.24it/s]





  8%|████████████████     

In [148]:
# json.loads(article_schema)['@graph'][0]#['datePublished']


Skip to content
Menu
主頁
數碼營銷課程
Blog
Jobs
找工作
發布工作
關於我們
會員註冊
FREE (7 個課程)
月費計劃 HKD$97 (50 個課程)
永久計劃 HKD$2,910 (50 個課程)
登入
點揀 SEO 公司? 究竟香港SEO公司 有幾多間？知唔知SEO 報價最平幾錢？
by ivan so
當你去香港 SEO公司拿取 SEO 報價的時候，會得到很多個不同的服務版本及SEO價錢，SEO最平價格我看過有 $3,000，相對昂貴的有 $30,000，收費相差可以有 10 倍之大，如果你需要一個平均數，SEO 費用平均需要 $8,000 左右，而且這個 SEO 費用是每個月的。
有些人會用 Facebook ads 及 Google ads (SEM) 作出比較，從而衡量 SEO 的價值。
香港 SEO 公司有多少間？
香港 SEO 公司有很多間，當你於 Google 輸入SEO優化公司或SEO服務的時候，我估沒有 100 間也有 70 間公司提供搜尋引擎最佳化服務，SEO Hong Kong 公司及 digital marketing 公司有 HKGSEO、Ringoli 及 Hellotoby 平台的SEO服務商。
一些是網上推廣公司的 SEO 優化服務只是其中一個服務，你要了解一下他們搜尋引擎優化 search engine optimization 服務的專業性。
坊間 SEO 報價單
搜尋引擎關鍵字困難程度SEO 網站優化費用 / 月Google HK高$1,000 / 關鍵字Google HK中$500 / 關鍵字Google HK低$200 / 關鍵字
Table Of Contents香港 SEO 公司有多少間？SEO服務的 8 個項目5 大 SEO 公司推薦的方法您需要 SEO 的幫助嗎？
SEO服務的 8 個項目
(1) 網站審計
SEO 公司會用一些搜尋引擎最佳化工具去審計你網站的及提供建議， SEO 工具包括 ahrefs、moz、semrush 及 woorank。
Moz bar: https://moz.com/products/pro/seo-toolbar (free)
(2) 關鍵字查詢
SEO 服務包括 Google 關鍵字查詢，他們會進行一些關鍵字建議，有一些是比較困難獲得的，有一

In [150]:
df = pd.DataFrame(collect, columns=['link', 'article_desc', 'article_keywords', 'article_content', 'article_published_time', 'article_modified_time', 'article_schema'])
print(df.shape)
df.head()


(50, 7)


,link,article_desc,article_keywords,article_content,article_published_time,article_modified_time,article_schema
0,https://www.da-vinci.com.tw/tw/blog/seo-cost,大家都在討論SEO關鍵字排名，市場報價非常紊亂，比網頁設計更沒有報價依據，加上很多人不懂SE...,,搜尋 搜尋 f TW TW CN 首頁 關於我們 公司介紹 專業團隊 服務項目 網頁設計 網...,,,
1,https://inboundmarketing.com.tw/seo/seo%E5%85%...,一個合格的SEO公司能為你服務的，而不是只是單純的排名與成效收費，傳統類型的SEO 公司更重...,,集客數據行銷 內容行銷搭配搜尋優化，讓客戶更懂你 數位行銷顧問服務 數位行銷顧問 電商行銷顧...,2018-01-14T00:52:34+00:00,2020-04-17T04:05:50+00:00,
2,https://www.yesharris.com/before-seo-agency/,即便你找台灣真正懂SEO的人、甚至是台灣最頂尖的專家來輔導你做 SEO，你的 SEO也不一定...,,關於Harris 文章分類 SEO教學 SEO 基礎教學 SEO 進階觀念 Search C...,2018-10-08,2020-08-27,
3,https://growthmarketing.tw/seo%E5%84%AA%E5%8C%...,,,首頁 關於零一 服務介紹 成長顧問諮詢 內容行銷案例 成長行銷知識庫 選單 首頁 關於零一 ...,2020-07-28T22:51:23+00:00,2020-09-07T15:38:48+00:00,
4,https://ranking.works/SEO%E6%95%99%E5%AD%B8/SE...,不同階段的企業可以藉由 SEO 達到什麼樣的效益、服務有哪些形式以及如何選擇適合的 SEO ...,,服務項目 SEO軟體 SEO教學部落格 關於我們 聯絡我們 註冊/登入 服務項目 SEO軟體...,,,


In [156]:
df.to_csv('{}_{}_articles_features.csv'.format(search_keyword, num), index=False)
